In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241204_02_MovieLens")
spark = SparkContext(conf=conf).getOrCreate()

24/12/04 14:42:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import os
directory = os.path.join(os.getcwd(),"data")
filename = "u.data"
filepath = os.path.join(directory, filename)
lines = spark.textFile("file:///"+filepath.replace("\\","/"))
lines

file:////home/lab12/src/data/u.data MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [3]:
lines.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [4]:
# 각 줄을 탭(\t)으로 분리
split_lines = lines.map(lambda line: line.split("\t"))

# 별점만 추출
ratings = split_lines.map(lambda fields: fields[2])  # 세 번째 필드

# 추출된 별점 확인
print(ratings.take(10))


['3', '3', '1', '2', '1', '4', '2', '5', '3', '3']


In [5]:
# 각 별점에 1을 할당하고, 키(별점)별로 합산
rating_counts = ratings.map(lambda rating: (rating, 1)).reduceByKey(lambda a, b: a + b)

# 별점별 카운트 확인
print(rating_counts.collect())


[('3', 27145), ('1', 6110), ('2', 11370), ('4', 34174), ('5', 21201)]


In [6]:
# 별점 기준으로 정렬
sorted_counts = rating_counts.sortByKey()

# 정렬된 결과 확인
print(sorted_counts.collect())


[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]


In [ ]:
#가장 많이 평가된 영화 찾기

In [8]:
# 영화 ID만 추출
movie_ids = split_lines.map(lambda fields: fields[1])

# 영화별 평가 횟수 카운트
movie_counts = movie_ids.map(lambda movie_id: (movie_id, 1)).reduceByKey(lambda a, b: a + b)

# 평가 횟수가 많은 영화 상위 10개
top_movies = movie_counts.takeOrdered(10, key=lambda x: -x[1])
print(top_movies)


[('50', 583), ('258', 509), ('100', 508), ('181', 507), ('294', 485), ('286', 481), ('288', 478), ('1', 452), ('300', 431), ('121', 429)]


In [9]:
# 1. 영화 ID와 별점 추출
movie_ratings = split_lines.map(lambda fields: (fields[1], float(fields[2])))

# 2. 3점 이상인 데이터만 필터링
filtered_ratings = movie_ratings.filter(lambda x: x[1] >= 3)

# 3. 영화별 총합과 평가 개수 계산
rating_totals_and_counts = filtered_ratings.mapValues(lambda rating: (rating, 1)) \
                                           .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

# 4. 평균 평점 계산
average_ratings = rating_totals_and_counts.mapValues(lambda x: x[0] / x[1])

# 5. 결과 확인 (평균 평점 상위 10개 영화)
top_rated_movies = average_ratings.takeOrdered(10, key=lambda x: -x[1])
print(top_rated_movies)


[('1306', 5.0), ('1189', 5.0), ('1500', 5.0), ('814', 5.0), ('1175', 5.0), ('1536', 5.0), ('1357', 5.0), ('1293', 5.0), ('1617', 5.0), ('247', 5.0)]


In [10]:
spark.stop()